# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [ ]:
!pip install kagglehub[pandas-datasets]

In [1]:
from azureml.core import Workspace, Experiment
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.train.automl import AutoMLConfig

from azureml.core.environment import Environment
from azureml.core.model import InferenceConfig
from azureml.core.model import Model
from azureml.core.webservice import AciWebservice, Webservice
from azureml.core import Dataset

from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

import pandas as pd

import os
# import kagglehub
# from azureml.data.dataset_profile import DataProfile
# from azureml.data.dataset_profile.run_config import DatasetProfileRunConfig


## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [3]:
ws = Workspace.from_config()

experiment_name = 'automl-exp'

experiment=Experiment(ws, experiment_name)

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = experiment.start_logging()

Workspace name: udacity_ws
Azure region: eastus
Subscription id: 051560d6-9344-4907-a9c5-057add5cf030
Resource group: udacity_rg


In [3]:
cluster_name = "aml-compute1"

try:
    compute_target = ComputeTarget(workspace=ws, name=cluster_name)
    print(f"Found existing compute target: {cluster_name}")
except ComputeTargetException:
    print(f"Creating new compute target: {cluster_name}")
    compute_config = AmlCompute.provisioning_configuration(
        vm_size="Standard_DS3_v2",
        min_nodes=0,
        max_nodes=4
    )

    compute_target = ComputeTarget.create(ws, cluster_name, compute_config)
    compute_target.wait_for_completion(show_output=True)

Found existing compute target: aml-compute1


In [5]:
def clean_df(df):

    ptd = pd.get_dummies(df.periodTypeDescription, "ptd")
    df.drop("periodTypeDescription", inplace=True, axis=1)
    df = df.join(ptd)

    dr = pd.get_dummies(df.dataRegion, "dr")
    df.drop("dataRegion", inplace=True, axis=1)
    df = df.join(dr)

    an = pd.get_dummies(df.areaName, "an")
    df.drop("areaName", inplace=True, axis=1)
    df = df.join(an)

    df.drop("title", inplace=True, axis=1)
    df.drop("areaDescription", inplace=True, axis=1)
    df.drop("cpiSourceDescription", inplace=True, axis=1)
    
    return df

In [13]:
dataset_name = 'denver-cpi'

try : 

    ds = Dataset.get_by_name(ws, dataset_name)
    print(f"Dataset '{dataset_name}' found and loaded.")
    
except Exception as e:
    print("downloading dataset")
    path = kagglehub.dataset_download("bls/denver-cpi")
    path = os.path.join(path, "Consumer_Price_Index_in_Denver.csv")
    datastore = ws.get_default_datastore()
    df = pd.read_csv(path)
    print("read dataset")
    print(df.shape)
    df = clean_df(df)
    print("cleaned dataset")

    ds = Dataset.Tabular.register_pandas_dataframe(
            dataframe = df, 
            name=dataset_name,
            description='CPI denver dataset',
            target=datastore
        )
    print("registered dataset")

    # profile_run_config = DatasetProfileRunConfig(compute_target=compute_target)
    # data_profile = DataProfile(profile_run_config)
    # profile_result = data_profile.profile(ds)
    # print("profiled dataset")


downloading dataset
read dataset
(4349, 17)
cleaned dataset
Validating arguments.
Arguments validated.
Validating arguments.
Arguments validated.
'overwrite' is set to True. Any file already present in the target will be overwritten.
Uploading files from '/tmp/tmphxw3t4bw' to 'managed-dataset/be019c32-5e39-4a1a-b127-1ff8c5ae7518/'
Copying 1 files with concurrency set to 1
Copied /tmp/tmphxw3t4bw/dataframe.parquet, file 1 out of 1. Destination path: https://udacityws6195221394.blob.core.windows.net/azureml-blobstore-c25467c7-a099-4be5-9286-b43241d0e0c0/managed-dataset/be019c32-5e39-4a1a-b127-1ff8c5ae7518/dataframe.parquet
Successfully uploaded file to datastore.
Creating and registering a new dataset.
Files copied=1, skipped=0, failed=0
Successfully created and registered a new dataset.
registered dataset


## AutoML Configuration

experiment_timeout_hours is set to low to limit cost <br> 
enable_early_stopping saves compute and avoids overtraining <br>
iteration_timeout_minutes avoids overly long-running iterations like deep ensembles <br>
max_concurrent_iterations is based on my compute target. 

In [14]:
automl_settings = {
    "experiment_timeout_hours" : 0.2,
    "enable_early_stopping" : True,
    "iteration_timeout_minutes" : 5,
    "max_concurrent_iterations" : 3,
}

automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task="regression",
    primary_metric="normalized_root_mean_squared_error",
    training_data = ds,
    label_column_name="cpi",
    n_cross_validations=3,
    compute_target=compute_target,
    **automl_settings)

In [15]:
remote_run = experiment.submit(automl_config)

Submitting remote run.


Experiment,Id,Type,Status,Details Page,Docs Page
automl-exp,AutoML_7d8d3092-e927-4148-90a6-69921bed2bea,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation


## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [ ]:
# from azureml.widgets import RunDetails
# RunDetails(remote_run).show()

In [49]:
experiment = Experiment(workspace=ws, name='automl-exp')
for run in experiment.get_runs():
    print(f"Run ID: {run.id} | Status: {run.status}")

Run ID: 1f27a776-0f23-47c2-8ca8-dcaf690fe016 | Status: Running
Run ID: eb56ee6d-296b-413f-b521-23d5df1b26da | Status: Running
Run ID: AutoML_7d8d3092-e927-4148-90a6-69921bed2bea | Status: Completed
Run ID: AutoML_198314eb-df85-45a2-9320-928a203f3bcf | Status: Canceled
Run ID: 945f3304-4943-468a-ba78-e91d3ec01a65 | Status: Running


## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [18]:
best_run = remote_run.get_best_child()

In [19]:
best_run

Experiment,Id,Type,Status,Details Page,Docs Page
automl-exp,AutoML_7d8d3092-e927-4148-90a6-69921bed2bea_34,azureml.scriptrun,Completed,Link to Azure Machine Learning studio,Link to Documentation


In [22]:
metrics = best_run.get_metrics()
for key, value in metrics.items():
    print(f"{key}: {value}")

median_absolute_error: 0.14155778017911422
normalized_root_mean_squared_log_error: 0.002402851691255647
explained_variance: 0.9996864204443777
normalized_median_absolute_error: 0.0002960363634786161
root_mean_squared_error: 1.3089292727085868
mean_absolute_percentage_error: 0.5299205139285355
r2_score: 0.999686094900372
spearman_correlation: 0.9998260321529827
mean_absolute_error: 0.43364264216952003
normalized_mean_absolute_error: 0.0009068663741031458
root_mean_squared_log_error: 0.009183380023524366
normalized_root_mean_squared_error: 0.0027373321441821474
predicted_true: aml://artifactId/ExperimentRun/dcid.AutoML_7d8d3092-e927-4148-90a6-69921bed2bea_34/predicted_true
residuals: aml://artifactId/ExperimentRun/dcid.AutoML_7d8d3092-e927-4148-90a6-69921bed2bea_34/residuals


## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [ ]:
model = best_run.register_model(model_name='automl_best_model', model_path='outputs/model.pkl')

In [ ]:
env = Environment.from_conda_specification(name="my-env",file_path="env.yaml")
inference_config = InferenceConfig(entry_script="score.py", environment=env)

deployment_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb= 4, auth_enabled = True)

service = Model.deploy(
    workspace = ws,
    name="automl-model-ws",
    models=[model],
    inference_config=inference_config,
    deployment_config=deployment_config,
    overwrite=True
)

service.wait_for_deployment(show_output=True)

TODO: In the cell below, send a request to the web service you deployed to test it.

In [40]:
import urllib.request
import json

data = {
  "Inputs": {
    "data": [
      {
        "stateFips": 0,
        "area": 0,
        "areaType": 0,
        "period": 0,
        "periodYear": 0,
        "periodType": 0,
        "type": 0,
        "source": 0,
        "percentChangeYear": 0,
        "percentChangeMonth": 0,
        "ptd_Annual": 0,
        "ptd_Monthly": 0,
        "ptd_Semi-Annual": 0,
        "dr_CO": 0,
        "dr_US": 0,
        "an_Denver/Boulder/Greeley": 0,
        "an_United States": 0
      }
    ]
  },
  "GlobalParameters": 1
}

body = str.encode(json.dumps(data))

url = 'http://218198b9-6ccc-4d0d-9619-0bcd0a37468c.eastus.azurecontainer.io/score'
api_key = os.getenv("AUTO_ML_API_KEY")
if not api_key:
    raise Exception("A key should be provided to invoke the endpoint")

headers = {'Content-Type':'application/json', 'Accept': 'application/json', 'Authorization':('Bearer '+ api_key)}
req = urllib.request.Request(url, body, headers)

b'{"Results": [9.953594814647328]}'


TODO: In the cell below, print the logs of the web service and delete the service

In [45]:
try:
    response = urllib.request.urlopen(req)
    result = response.read()
    print(result)
except urllib.error.HTTPError as error:
    print("The request failed with status code: " + str(error.code))
    print(error.info())
    print(error.read().decode("utf8", 'ignore'))

b'{"Results": [9.953594814647328]}'


In [46]:
service = Webservice(workspace=ws, name='automl-model-ws')
print(service)

AciWebservice(workspace=Workspace.create(name='udacity_ws', subscription_id='051560d6-9344-4907-a9c5-057add5cf030', resource_group='udacity_rg'), name=automl-model-ws, image_id=None, image_digest=None, compute_type=ACI, state=Healthy, scoring_uri=http://218198b9-6ccc-4d0d-9619-0bcd0a37468c.eastus.azurecontainer.io/score, tags={}, properties={'runId': 'AutoML_7d8d3092-e927-4148-90a6-69921bed2bea_34', 'hasInferenceSchema': 'True', 'hasHttps': 'False', 'authEnabled': 'True'}, created_by={'userObjectId': 'fe75403a-f3c0-4149-ba5e-7607c21d4f2b', 'userPuId': '10032004C429A1AA', 'userIdp': 'live.com', 'userAltSecId': '1:live.com:00037FFF9AAB24E8', 'userIss': 'https://sts.windows.net/a54ddfa5-3d29-437b-a420-b882e37e67e2/', 'userTenantId': 'a54ddfa5-3d29-437b-a420-b882e37e67e2', 'userName': 'Aseer Ahmad Ansari', 'upn': None})


In [ ]:
service.delete()
print("Web service deleted.")

**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.
